# Chapter 2: Decision Trees in Depth
This chapter covers the following main topics:
- Introducing decision trees with XGBoost
- Exploring decision trees
- Contrasting variance and bias
- Tuning decision tree hyper parameters
- Predicting heart disease: a case study

## Introducing decision trees with XGBoost
XGBoost is an ensemble method: it is composed of different machine learning models that combine to work well together. The individual models in the ensemble are called *base learners*. The most common type of base learner is the decision tree. Decision trees split data by asking questions about the columns. Unfortunately, they are prone to overfitting, meaning they can predict the training set very well but generalize poorly. XGBoost and random forests aggregate the predictions of many trees. This reduces overfitting and improves performance. 

## Exploring decision trees
Decision trees split data into branches, which are followed down to leaves, where predictions are made.

In [24]:
# Let's build a decision tree
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [25]:
df_census = pd.read_csv('census_cleaned.csv')

In [26]:
# Declare predictor and target columns
X = df_census.iloc[:, :-1]
y = df_census.iloc[:, -1]

In [27]:
# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, random_state=1)

In [28]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [29]:
# Initialize the decision tree classifier
clf = DecisionTreeClassifier(random_state=2)

# Fit the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Compare predictions to the test set
print(accuracy_score(y_pred, y_test))

0.8212750276378823


### Inside a decision tree

Decision trees are composed of nodes and leaves. Nodes ask questions about the data. Leaves make predictions. The first node (at the top of the tree) is called the *root node*. The last nodes are called *terminal nodes* or *leaves*. The nodes in between are called *internal nodes*.

The Gini criterion is the error method the decision tree uses to decide how splits should be made. The goal is to find a split that leads to the lowest error. A gini index of `0` means 0 error, and a gini index of `0.5` indicates a node where the elements are evenly split between prediction classes.

The root example in the book is:
```
gini = 0.364
samples = 24420
value = [18575, 5845]
class = 0
```
The `value` line indicates that there are 18,575 examples of class `0` and 5,845 examples of class `1`. The `class` line indicates that the majority class is `0`. The `gini` line indicates that the error is `0.364`.

It's possible to have a tree with only one split, called a *stump*. Stumps aren't powerful predictors in themselves, but can be powerful when used as boosters.

## Contrasting variance and bias
Bias is a mathematical term that comes from estimating the error when applying the model to a real-life problem. The bias of a straight line (e.g. trying to fit a linear regression to a quadratic function) is high, indicating underfitting. The model is too simple to capture the complexity of the data. 

Variance is a mathematical term indicating how much the model will change given a different set of training data. A model with high variance will change a lot given different training data. This indicates overfitting: the model will perform well on the training data but will generalize poorly to new data.

Low variance means that the model will not change much given different training data. Low bias indicates that the error when applying the model to a real-world situation will not be too high. The goal is to find a model that balances low bias and low variance.

## Tuning decision tree hyper parameters
Whereas parameters are adjusted when the model is being tuned, hyperparameters are adjusted before the model is trained. Hyperparameters are set before training and remain constant during training. The best way to learn about hyperparameters is through experimentation. Although there are some general rules, the best way to find the best hyperparameters is to try different values and see what works best.

In [47]:
# Let's look at hyperparameters using a decision tree regressor
df_bikes = pd.read_csv("bike_rentals_cleaned.csv")
X_bikes = df_bikes.iloc[:, :-1]
y_bikes = df_bikes.iloc[:, -1]

# Split data into train and test sets
# The book forgot this line, but I shouldn't have.
X_train, X_test, y_train, y_test = train_test_split(X_bikes, y_bikes, random_state=2)

In [48]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

# Initialize the decision tree regressor
reg = DecisionTreeRegressor(random_state=2)
scores = cross_val_score(reg, X_bikes, y_bikes, scoring='neg_mean_squared_error',cv=5)

# Take the square root of the scores
rmse = np.sqrt(-scores)
print('RMSE mean: %0.2f' % (rmse.mean()))

RMSE mean: 1233.36


In [49]:
# This is worse than the scores from Linear Regression and XGBoost in chapter 1. Is the variance too high?
# Check by seeing how well the decision tree makes predictions on the training set alone.

reg = DecisionTreeRegressor()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_train)

from sklearn.metrics import mean_squared_error
reg_mse = mean_squared_error(y_train, y_pred)
reg_rmse = np.sqrt(reg_mse)
reg_rmse

0.0

In [50]:
# The model fit the training set nearly perfectly, so it has high variance.
# Hyperparameters may help. 
# The book goes into quite a bit of detail about DecisionTreeRegressor hyperparameters here.
# Try using GridSearchCV to find the best number for max_depth, which is the number of split iterations.
from sklearn.model_selection import GridSearchCV
params = {'max_depth':[None, 2, 3, 4, 6, 8, 10, 20]}

# Initialize a DecisionTreeRegressor and place it inside of GridSearchCV with params and a scoring metric.
reg = DecisionTreeRegressor(random_state=2)
grid_reg = GridSearchCV(reg, params, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
grid_reg.fit(X_train, y_train)

# View best hyperparameters
best_params = grid_reg.best_params_
print('Best params: ', best_params)

Best params:  {'max_depth': 6}


In [51]:
# View the training score
best_score = np.sqrt(-grid_reg.best_score_)
print(f"Training score: {best_score}")

Training score: 951.3984508554636


In [52]:
best_model = grid_reg.best_estimator_
y_pred = best_model.predict(X_test)
rmse_test = mean_squared_error(y_test, y_pred) ** 0.5
print('Test score: {:.3f}'.format(rmse_test))

Test score: 864.670


In [53]:
# min_samples_leaf is another hyperparameter designed to reduce overfitting.
# The default min_samples_leaf=1, meaning that leaves may consist of unique samples, leading to overfitting.
# Use an iterative approach to determine the best parameters and test scores using GridSearchCV
def grid_search(params, reg=DecisionTreeRegressor(random_state=2)):
    grid_reg = GridSearchCV(reg, params, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
    grid_reg.fit(X_train, y_train)
    
    best_params = grid_reg.best_params_
    print('Best params: ', best_params)
    
    best_score = np.sqrt(-grid_reg.best_score_)
    print('Training score: {:.3f}'.format(best_score))
    
    y_pred = grid_reg.predict(X_test)
    rmse_test = mean_squared_error(y_test, y_pred) ** 0.5
    print('Test score: {:.3f}'.format(rmse_test))

In [54]:
# It's helpful to know the size of the training set when choosing the range of hyperparameters
X_train.shape

(548, 12)

In [56]:
# Knowing the number of samples helps determine reasonable values for min_samples_leaf. 
# The book sounds very confident, but how can I figure this out for myself?
grid_search(params={'min_samples_leaf': [1, 2, 4, 6, 8, 10, 20, 30]})

Best params:  {'min_samples_leaf': 8}
Training score: 896.083
Test score: 855.620


In [57]:
# Test score better than training score means that variance has been reduced.
# Let's try grid_search with min_samples_leaf and _max_depth together?
# Recall that max_depth = 6 was previously the best.
grid_search(params={'max_depth': [6, 7, 8, 9, 10], 'min_samples_leaf': [3, 5, 7, 9]})

Best params:  {'max_depth': 9, 'min_samples_leaf': 7}
Training score: 888.905
Test score: 878.538


The book covers several other hyperparameters worth reviewing to reduce variance: 
- `max_leaf_nodes`: similar to `min_samples_leaf`, except it specifies the total number of leaves. `max_leaf_nodes=10` means the model can have 10 or fewer leaves.
- `max_features`: chooses from a select number of features each round rather than considering every possible feature for a split.
- `min_samples_split`: provides a limit to the number of samples required before a split can be made.
- `splitter`: tells the model how to select the feature to split each branch. `best` selects the feature that results in the greatest information gain.
- `criterion`: the scoring method for splits. Options for decision tree regressors are `mse` (mean squared error), `friedman_mse` (MSE including Friedman's adjustment) and `mae` (mean absolute error). Default is `mse`. For classifiers, `gini` and `entropy` usually give similar results.
- `min_impurity_decrease`: results in a split when the impurity is greater than or equal to this value. A tree with 100% accuracy would have an impurity of 0.0; a tree with 80% accuracy would have an impurity of 0.20. Impurity should gradually decrease throughout the tree-building process.
- `min_weight_fraction_leaf`: minimum weighted fraction of the total weights required to be a leaf. Practically speaking, assuming equal weights, a restriction of 1% (`0.01`) would require at least five of the 500 samples to be a leaf.
- `ccp_alpha`: designed for pruning after the tree has been built. See the documentation on minimal cost complexity pruning in the `scikit-learn` docs.

## Predicting heart disease: a case study

In [58]:
df_heart = pd.read_csv('heart_disease.csv')
df_heart.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [59]:
# Split the data into training and test sets
X = df_heart.iloc[:,:-1]
y = df_heart.iloc[:,-1]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [60]:
# Look at the baseline model before implementing hyperparameters
model = DecisionTreeClassifier(random_state=2)
scores = cross_val_score(model, X, y, cv=5)
print('Accuracy:', np.round(scores,2))
print('Accuracy mean: %0.2f' % (scores.mean()))

Accuracy: [0.74 0.85 0.77 0.73 0.7 ]
Accuracy mean: 0.76


In [61]:
# RandomizedSearchCV is a wonderful alternative to GridSearchCV when tuning many hyperparameters.
# Instead of trying all hyperparameters, it tries a random number of combinations.
# It's not exhaustive, but great for finding the best combinations in limited time.
from sklearn.model_selection import RandomizedSearchCV


# Write a function to use RandomizedSearchCV to return the best model and scores
def randomized_search_clf(params, runs=20, clf=DecisionTreeClassifier(random_state=2)):
    rand_clf = RandomizedSearchCV(
        clf, params, n_iter=runs, cv=5, n_jobs=-1, random_state=2
    )
    rand_clf.fit(X_train, y_train)
    best_model = rand_clf.best_estimator_
    best_score = rand_clf.best_score_
    print("Training score: {:.3f}".format(best_score))
    y_pred = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Test score: {:.3f}".format(accuracy))
    return best_model

In [62]:
# Experiment with hyperparameters
randomized_search_clf(params={'criterion':['entropy',
'gini'],'splitter':['random', 'best'], 'min_weight_fraction_leaf':[0.0, 0.0025, 0.005, 0.0075, 0.01],'min_samples_split':[2, 3, 4, 5, 6, 8, 10],
'min_samples_leaf':[1, 0.01, 0.02, 0.03, 0.04],
'min_impurity_decrease':[0.0, 0.0005, 0.005, 0.05, 0.10, 0.15,
0.2],'max_leaf_nodes':[10, 15, 20, 25, 30, 35, 40, 45, 50,
None],'max_features':['auto', 0.95, 0.90, 0.85, 0.80, 0.75,
0.70],'max_depth':[None, 2,4,6,8],
'min_weight_fraction_leaf':[0.0, 0.0025, 0.005, 0.0075, 0.01,
0.05]})

Training score: 0.798
Test score: 0.855


DecisionTreeClassifier(criterion='entropy', max_depth=8, max_features=0.8,
                       max_leaf_nodes=45, min_samples_leaf=0.04,
                       min_samples_split=10, min_weight_fraction_leaf=0.05,
                       random_state=2)

In [63]:
# Let's narrow the range to improve hyperparameters.
# Use a baseline of max_depth = 8 to narrow the range from 7 to 9.
# Another strategy is to stop checking hyperparameters whose defaults are fine. Anything not in the returned model can be left alone.
# Try a new hyperparameter range with an increase of 100 runs.
randomized_search_clf(
    params={
        "max_depth": [None, 6, 7],
        "max_features": ["auto", 0.78],
        "max_leaf_nodes": [45, None],
        "min_samples_leaf": [1, 0.035, 0.04, 0.045, 0.05],
        "min_samples_split": [2, 9, 10],
        "min_weight_fraction_leaf": [0.0, 0.05, 0.06, 0.07],
    },
    runs=100,
)

Training score: 0.802
Test score: 0.868


DecisionTreeClassifier(max_depth=7, max_features=0.78, max_leaf_nodes=45,
                       min_samples_leaf=0.045, min_samples_split=9,
                       min_weight_fraction_leaf=0.06, random_state=2)

In [66]:
# We must run the new model through cross_val_clf to get a true baseline.
model = DecisionTreeClassifier(
    class_weight=None,
    criterion="gini",
    max_depth=7,
    max_features=0.78,
    max_leaf_nodes=45,
    min_impurity_decrease=0.0,
    min_samples_leaf=0.045,
    min_samples_split=9,
    min_weight_fraction_leaf=0.06,
    random_state=2,
    splitter="best",
)

# Obtain cross-validation scores
scores = cross_val_score(model, X, y, cv=5)

# Display accuracy
print("Accuracy:", np.round(scores, 2))

# Display mean accuracy
print("Accuracy mean: %0.2f" % (scores.mean()))

Accuracy: [0.82 0.9  0.8  0.8  0.78]
Accuracy mean: 0.82


In [67]:
# Decision trees come with an attribute feature_importances_ to communicate the important features
# Define the model using the best hyperparameters
best_clf = DecisionTreeClassifier(
    class_weight=None,
    criterion="gini",
    max_depth=9,
    max_features=0.8,
    max_leaf_nodes=47,
    min_impurity_decrease=0.0,
    min_samples_leaf=1,
    min_samples_split=8,
    min_weight_fraction_leaf=0.05,
    random_state=2,
    splitter="best",
)

# Fit the model to the dataset
best_clf.fit(X, y)

DecisionTreeClassifier(max_depth=9, max_features=0.8, max_leaf_nodes=47,
                       min_samples_split=8, min_weight_fraction_leaf=0.05,
                       random_state=2)

In [68]:
# Let's see the most important features
best_clf.feature_importances_

array([0.04830121, 0.04008887, 0.47546568, 0.        , 0.        ,
       0.        , 0.        , 0.00976578, 0.        , 0.02445397,
       0.02316427, 0.1774694 , 0.20129082])

In [69]:
# Zip the columns along with the most important features, then display them in reverse order
feature_dict = dict(zip(X.columns, best_clf.feature_importances_))

import operator

# Sort dict by values as a list of tuples
sorted(feature_dict.items(), key=operator.itemgetter(1), reverse=True)[0:3]

[('cp', 0.47546567857183675),
 ('thal', 0.20129082387838435),
 ('ca', 0.1774694042213901)]

The three most important features are `cp` (chest pain type), `thalach` (maximum heart rate achieved), and `ca` (number of major vessels colored by fluoroscopy). These numbers may be interpreted as their explanation of variance: `cp` accounts for 48% of the variance, which is more than `thal` and `ca` combined. You can communicate with doctors and nurses by telling them that this model predicts if a patient has a heart disease with 82% accuracy using chest pain, maximum heart rate, and fluoroscopy as the three most important characteristics.